In [25]:
import pandas as pd
import scipy

from scipy.stats import mannwhitneyu
import requests
import json
from urllib.parse import urlencode

In [2]:
# 1 Task Retention
df_auth = pd.read_csv('~/shared/problem1-auth_data.csv', sep=';')
df_reg = pd.read_csv('~/shared/problem1-reg_data.csv', sep=';')

In [3]:
# define, what dataframe is the biggest to use the correct join and not to loose a part of data

(9601013, 2)

In [4]:
df_reg.shape

(1000000, 2)

In [26]:
# define the definition, which calculate retention. the number of periods is stored in a variable, 
# to change it anytime.
def count_retention(df_auth, df_reg, count_periods=20):
    from datetime import timedelta
    df = df_auth\
        .merge(df_reg, on='uid', how='left')
    df['date_reg'] = pd.to_datetime(df.reg_ts, unit='s')
    # from max date subtract periods quantity, required by task
    min_date = df.date_reg.max() - timedelta(days=count_periods)
    # take only last count_periods of dates
    df = df[df.date_reg > min_date]
    df['date_auth'] = pd.to_datetime(df.auth_ts, unit='s').dt.strftime('%Y-%m-%d')
    df['date_reg'] = df.date_reg.dt.strftime('%Y-%m-%d')
    cohorts = df.groupby(['date_reg', 'date_auth'])\
        .agg({'uid': 'nunique'})\
        .sort_values(['date_reg', 'date_auth'])
    
    # define the function, which приводит дату(строка) к datetime
    def parse_date(x): 
        return pd.to_datetime(x, format="%Y-%m-%d")

    # define the function, which calculate a different between dates
    def month_diff(a, b):
        a = parse_date(a)
        b = parse_date(b)
        return 12 * 30 * (a.year - b.year) + 30 * (a.month - b.month) + (a.day - b.day)

    # define the function, which calculate the difference for each cohort from the start of cohort(first month of purchase)
    # to (each)purchase month
    def cohort_period(df):
        df['CohortPeriod'] = month_diff(df.index.get_level_values('date_auth'), df.index.get_level_values('date_reg'))
        df['date'] = df['uid'] / df['uid'][0]
        return df
    cohorts_def = cohorts.groupby(level=0).apply(cohort_period)
    # create a table and return it
    return cohorts_def[cohorts_def.CohortPeriod <= count_periods]\
        .reset_index().pivot_table(index='date_reg', columns='CohortPeriod', values='date')

In [6]:
# run example
count_retention(df_auth, df_reg, 10)

CohortPeriod,0,1,2,3,4,5,6,7,8,9,10
date_reg,,,,,,,,,,,
2020-09-13,1.0,0.008503,0.039116,0.045918,0.051020,0.066327,0.078231,0.045918,0.049320,0.037415,0.037415
2020-09-14,1.0,0.022236,0.038295,0.045090,0.053119,0.064855,0.069796,0.063002,0.044472,0.029030,NaN
2020-09-15,1.0,0.022181,0.040665,0.045595,0.060382,0.056069,0.069008,0.056685,0.024646,NaN,NaN
2020-09-16,1.0,0.019077,0.042462,0.046154,0.046769,0.067692,0.069538,0.033846,NaN,NaN,NaN
2020-09-17,1.0,0.016585,0.042998,0.045455,0.054668,0.062654,0.042383,NaN,NaN,NaN,NaN
2020-09-18,1.0,0.019018,0.045399,0.042331,0.052147,0.037423,NaN,NaN,NaN,NaN,NaN
2020-09-19,1.0,0.018360,0.035496,0.047124,0.034884,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-20,1.0,0.024450,0.043399,0.023839,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-21,1.0,0.018926,0.029915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# 2 task 
url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
link = 'https://disk.yandex.ru/d/12gk_ATEeBZm4A'

response = requests.get(url + urlencode({'public_key': link}))

file_link = json.loads(response.text)['href']
df_2 = pd.read_csv(file_link, sep=';')

In [8]:
# slit test and control group and calculate ARPU
df_a = df_2.query("testgroup=='a'")

In [9]:
ARPU_a = df_a.revenue.sum() / df_a.user_id.nunique()

In [10]:
df_b = df_2.query("testgroup=='b'")

In [11]:
ARPU_b = df_b.revenue.sum() / df_b.user_id.nunique()

In [12]:
ARPU_a

25.413719736965806

In [13]:
ARPU_b

26.75128659327863

In [14]:
# ARPU in a test group higher, then in control.Prove, these difference are statistically significant

In [15]:
# 1 step : need to check, that data is normally distributed in each group.
# so we we apply the Shapiro-Wilk criterion. 
# as a null hypothesis we take, that distribution is normal. alternative is that distribution is different that normal

In [16]:
a_sample = df_a.revenue.sample(1000, random_state=17)

In [17]:
b_sample = df_b.revenue.sample(1000, random_state=17)

In [18]:
scipy.stats.shapiro(a_sample)

ShapiroResult(statistic=0.06614720821380615, pvalue=0.0)

In [19]:
scipy.stats.shapiro(b_sample)

ShapiroResult(statistic=0.06715953350067139, pvalue=0.0)

In [20]:
# in both groups the distrivution is not normal because p value =0.0
# we reject the null hypothesis about the normality of the distribution

In [21]:
# step 2 since the distribution is not normak, we we apply Mann-Whitney U-test(non-parmetric analogue of the t-test)

In [22]:
# as a null hypothesis we take, that there is no difference between average in the general population (ARPU_a=ARPU_b)
# as an alternative we take, that the averages in the population are not equal,
# that means (ARPU_b is really higher,then ARPU_a)

In [23]:
mannwhitneyu(df_a.revenue, df_b.revenue)

MannwhitneyuResult(statistic=20491259376.0, pvalue=0.06269701316074398)

In [24]:
# as p-value is approximately 0.06, we cannot reject the null hypothesis. 
# that means that there is no statistically differenceт between ARPU_a and ARPU_b
# and that means, that ARPU mean is not statistically significantly different

In [ ]:
3 task
1.What metrics can be used to evaluate the results of the last past event?

-At first, I would calculate retention to make sure that people return to the game more often
-I will also calculate DAU to see the impact on the number of players on any given day
-I will calculate the CSAT customer satisfaction score in order to understand that people like the event
-It will also be useful to know the ASL average session length to look at the involvement in the game

We proceed from the hypothesis that players who spend more time in the game and enter the game more often
more involved in the gameplay - more likely to make an in-game purchase

2. Suppose, that in another event, we have complicated the mechanics of events so that for each unsuccessful attempt
completing a level, the player will roll back several levels. Will the set of performance evaluation metrics change?
If so, how?

We need to make sure players don't play worse because of a tougher event.

-For example, pay attention to the DAU, do not become less players every day?!
-You also need to look at how the average session duration has changed. It could both decrease when the player quits after several failed attempts, and increase when the player spends more time trying to complete the level 
